In [17]:
import pandas as pd
import numpy as np

In [21]:
data = pd.read_csv('Kershaw_Ff_2020.csv')
sc_fix = data[['release_speed', 'pitch_type', 'release_spin_rate', 'release_extension', 'vy0', 'vx0', 'vz0', 'ax', 'az', 'ay']]

In [22]:
print(sc_fix.head())

   release_speed pitch_type  release_spin_rate  release_extension         vy0  \
0           90.0         FF             2526.0                6.3 -130.949396   
1           90.4         FF             2454.0                6.3 -131.636192   
2           90.2         FF             2498.0                6.5 -131.032247   
3           90.5         FF             2412.0                6.2 -131.611386   
4           90.1         FF             2452.0                6.3 -130.982624   

        vx0        vz0        ax         az         ay  
0 -3.856777  -6.887478 -2.873058 -12.588795  27.050115  
1 -1.589323  -5.907190 -1.929007 -13.788574  28.007695  
2 -2.049258 -10.061238 -4.190430 -11.207077  28.299466  
3 -2.463220  -7.547582 -1.389470 -12.964300  27.527655  
4 -5.580918  -5.788556 -5.249012 -12.638718  29.685625  


In [23]:
K = 5.383E-03
z_constant = 32.174

# sc = pd.DataFrame()  # Assuming 'sc' is your input DataFrame

# sc_fix = sc.copy()

sc_fix['yR'] = 60.5 - sc_fix['release_extension']
sc_fix['tR'] = (-sc_fix['vy0'] - np.sqrt(sc_fix['vy0']**2 - 2 * sc_fix['ay'] * (50 - sc_fix['yR']))) / sc_fix['ay']
sc_fix['vxR'] = sc_fix['vx0'] + sc_fix['ax'] * sc_fix['tR']
sc_fix['vyR'] = sc_fix['vy0'] + sc_fix['ay'] * sc_fix['tR']
sc_fix['vzR'] = sc_fix['vz0'] + sc_fix['az'] * sc_fix['tR']

sc_fix['tf'] = (-sc_fix['vyR'] - np.sqrt(sc_fix['vyR']**2 - 2 * sc_fix['ay'] * (sc_fix['yR'] - 17/12))) / sc_fix['ay']

sc_fix['vxbar'] = (2 * sc_fix['vxR'] + sc_fix['ax'] * sc_fix['tf']) / 2
sc_fix['vybar'] = (2 * sc_fix['vyR'] + sc_fix['ay'] * sc_fix['tf']) / 2
sc_fix['vzbar'] = (2 * sc_fix['vzR'] + sc_fix['az'] * sc_fix['tf']) / 2

sc_fix['vbar'] = np.sqrt(sc_fix['vxbar']**2 + sc_fix['vybar']**2 + sc_fix['vzbar']**2)

sc_fix['adrag'] = -(sc_fix['ax'] * sc_fix['vxbar'] + sc_fix['ay'] * sc_fix['vybar'] + (sc_fix['az'] + z_constant) * sc_fix['vzbar']) / sc_fix['vbar']

sc_fix['amagx'] = sc_fix['ax'] + sc_fix['adrag'] * sc_fix['vxbar'] / sc_fix['vbar']
sc_fix['amagy'] = sc_fix['ay'] + sc_fix['adrag'] * sc_fix['vybar'] / sc_fix['vbar']
sc_fix['amagz'] = sc_fix['az'] + sc_fix['adrag'] * sc_fix['vzbar'] / sc_fix['vbar'] + z_constant

sc_fix['amag'] = np.sqrt(sc_fix['amagx']**2 + sc_fix['amagy']**2 + sc_fix['amagz']**2)

sc_fix['Cl'] = sc_fix['amag'] / (K * sc_fix['vbar']**2)
sc_fix['s'] = 0.166 * np.log(0.336 / (0.336 - sc_fix['Cl']))
sc_fix['spinT'] = 78.92 * sc_fix['s'] * sc_fix['vbar']
sc_fix['spin_efficiency'] = sc_fix['spinT'] / sc_fix['release_spin_rate']


C:\Users\logan.kniss\AppData\Local\Temp\ipykernel_17432\729600482.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sc_fix['yR'] = 60.5 - sc_fix['release_extension']
C:\Users\logan.kniss\AppData\Local\Temp\ipykernel_17432\729600482.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sc_fix['tR'] = (-sc_fix['vy0'] - np.sqrt(sc_fix['vy0']**2 - 2 * sc_fix['ay'] * (50 - sc_fix['yR']))) / sc_fix['ay']
C:\Users\logan.kniss\AppData\Local\Temp\ipykernel_17432\729600482.py:10: SettingWithCopyWarning: 
A value is t

In [25]:
sc_fix.head()
sc_fix.to_csv('sc_fix.csv')